In [1]:
import tensorflow as tf
import numpy as np
import os

In [13]:
with tf.variable_scope("outside",reuse=tf.AUTO_REUSE):
    x = tf.get_variable(name='x',initializer=tf.truncated_normal([2,3])).eval()
    y = tf.get_variable(name='y',initializer=tf.truncated_normal([7,4,5])).eval()

In [3]:
for var in tf.trainable_variables():
    shape = var.get_shape()
    print(shape)
    pd = 1
    for x in shape:
        pd*=x.value
    print(pd)
    print("____")

(2, 3)
6
____
(7, 4, 5)
140
____


In [14]:
with tf.variable_scope("first",reuse=tf.AUTO_REUSE):
    a = tf.get_variable("a",initializer=tf.truncated_normal([4,4])).eval()
    b = tf.get_variable("b",initializer = tf.truncated_normal([5,6])).eval()

In [5]:
def get_scope(l,i):
    if len(l)-1 == i:
        return tf.get_variable(name=l[i][:-2]).eval()
    with tf.variable_scope(l[i],reuse=True):
        get_scope(l,i+1)

In [15]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
store=dict()
for var in tf.trainable_variables():
    l=var.name.split('/')
    store[var.name] = get_scope(l,0)

C:\Users\k_tej\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [16]:
store

{'outside/x:0': None,
 'outside/y:0': None,
 'first/a:0': None,
 'first/b:0': None}